In [1]:
import folium
import pandas as pd
import numpy as np

In [2]:
f = folium.Figure()

center_lat, center_lon = 36.63145, 138.1881
# center_lat, center_lon = 80.0, -30.0

m = folium.Map([center_lat, center_lon], zoom_start=5).add_to(f)
folium.Marker(location=[center_lat, center_lon]).add_to(m)
m

In [3]:
N = 19
arr = np.arange(N)

radius = 0.0003

circle_lat = radius * np.sin(arr / N * np.pi * 2) + center_lat
circle_lon = radius * np.cos(arr / N * np.pi * 2) + center_lon

circle_lat_lon = np.stack([circle_lat, circle_lon], axis=1)

df = pd.DataFrame(circle_lat_lon, columns=["lat", "lon"])
df

,lat,lon
0,36.631450,138.188400
1,36.631547,138.188384
2,36.631634,138.188337
3,36.631701,138.188264
4,36.631741,138.188174
5,36.631749,138.188075
6,36.631725,138.187979
7,36.631671,138.187897
8,36.631593,138.187836
9,36.631499,138.187804


In [4]:
f = folium.Figure()

m = folium.Map([center_lat, center_lon], zoom_start=17).add_to(f)
for i in arr:
    folium.Marker(location=[df["lat"][i], df["lon"][i]]).add_to(m)
m

In [5]:
df.to_csv("csv/circle_sampe_n%d_lat%.4f.csv" % (N, center_lat), index=False)

In [6]:
df_vector = df[1:].copy()
df_vector["lat_prev"] = df[:-1]["lat"].tolist()
df_vector["lon_prev"] = df[:-1]["lon"].tolist()
df_vector["delta_lat"] = df_vector["lat"] - df_vector["lat_prev"]
df_vector["delta_lon"] = df_vector["lon"] - df_vector["lon_prev"]
df_vector["theta"] = np.arctan2(df_vector["delta_lat"], df_vector["delta_lon"])
df_vector

,lat,lon,lat_prev,lon_prev,delta_lat,delta_lon,theta
1,36.631547,138.188384,36.631450,138.188400,0.000097,-0.000016,1.736143
2,36.631634,138.188337,36.631547,138.188384,0.000087,-0.000047,2.066837
3,36.631701,138.188264,36.631634,138.188337,0.000067,-0.000073,2.397531
4,36.631741,138.188174,36.631701,138.188264,0.000040,-0.000090,2.728225
5,36.631749,138.188075,36.631741,138.188174,0.000008,-0.000098,3.058919
6,36.631725,138.187979,36.631749,138.188075,-0.000024,-0.000096,-2.893572
7,36.631671,138.187897,36.631725,138.187979,-0.000054,-0.000083,-2.562878
8,36.631593,138.187836,36.631671,138.187897,-0.000078,-0.000061,-2.232184
9,36.631499,138.187804,36.631593,138.187836,-0.000093,-0.000032,-1.901490
10,36.631401,138.187804,36.631499,138.187804,-0.000099,0.000000,-1.570796


In [7]:
def compute_theta_delta(x):
    x = abs(x)
    if (x > 180):
        x = 360 - x
    return x

In [8]:
df_direction = df_vector[1:][["theta"]].copy()
df_direction["theta_prev"] = df_vector["theta"][:-1].tolist()
df_direction["delta_theta"] = np.rad2deg(df_direction["theta"] - df_direction["theta_prev"]).apply(compute_theta_delta)
df_direction

,theta,theta_prev,delta_theta
2,2.066837,1.736143,18.947368
3,2.397531,2.066837,18.947368
4,2.728225,2.397531,18.947368
5,3.058919,2.728225,18.947368
6,-2.893572,3.058919,18.947368
7,-2.562878,-2.893572,18.947368
8,-2.232184,-2.562878,18.947368
9,-1.901490,-2.232184,18.947368
10,-1.570796,-1.901490,18.947368
11,-1.240102,-1.570796,18.947368


In [9]:
df_direction["delta_theta"].sum()

322.10526315364325

In [10]:
def compute_arc_length(lat1, lat2, lon1, lon2):
    """
    緯度経度から距離を計算 (単位球)
    """
    return np.arccos(np.sin(lat1) * np.sin(lat2) + np.cos(lat1) * np.cos(lat2) * np.cos(lon2 - lon1))

In [11]:
df_3points = df[:-2].copy().rename(columns={"lat": "lat1", "lon": "lon1"})
df_3points["lat2"] = df[1:-1]["lat"].tolist()
df_3points["lon2"] = df[1:-1]["lon"].tolist()
df_3points["lat3"] = df[2:]["lat"].tolist()
df_3points["lon3"] = df[2:]["lon"].tolist()
df_3points = np.deg2rad(df_3points)
c = compute_arc_length(df_3points["lat1"], df_3points["lat2"], df_3points["lon1"], df_3points["lon2"])
b = compute_arc_length(df_3points["lat2"], df_3points["lat3"], df_3points["lon2"], df_3points["lon3"])
a = compute_arc_length(df_3points["lat3"], df_3points["lat1"], df_3points["lon3"], df_3points["lon1"])

df_3points["a"] = a
df_3points["b"] = b
df_3points["c"] = c

df_3points["alpha"] = np.arccos((np.cos(a) - np.cos(b) * np.cos(c)) / (np.sin(b) * np.sin(c)))

# 表示法の変更
pd.options.display.precision = 2

df_3points

,lat1,lon1,lat2,lon2,lat3,lon3,a,b,c,alpha
0,0.64,2.41,0.64,2.41,0.64,2.41,3.34e-06,1.65e-06,1.72e-06,2.87
1,0.64,2.41,0.64,2.41,0.64,2.41,3.16e-06,1.55e-06,1.65e-06,2.83
2,0.64,2.41,0.64,2.41,0.64,2.41,2.95e-06,1.44e-06,1.55e-06,2.79
3,0.64,2.41,0.64,2.41,0.64,2.41,2.77e-06,1.39e-06,1.44e-06,2.74
4,0.64,2.41,0.64,2.41,0.64,2.41,2.73e-06,1.41e-06,1.39e-06,2.73
5,0.64,2.41,0.64,2.41,0.64,2.41,2.85e-06,1.49e-06,1.41e-06,2.76
6,0.64,2.41,0.64,2.41,0.64,2.41,3.06e-06,1.60e-06,1.49e-06,2.81
7,0.64,2.41,0.64,2.41,0.64,2.41,3.26e-06,1.69e-06,1.60e-06,2.85
8,0.64,2.41,0.64,2.41,0.64,2.41,3.38e-06,1.72e-06,1.69e-06,2.87
9,0.64,2.41,0.64,2.41,0.64,2.41,3.38e-06,1.69e-06,1.72e-06,2.87


In [12]:
df_3points["delta_theta"] = 180 - np.rad2deg(df_3points[["alpha"]])
df_3points[["delta_theta"]]

,delta_theta
0,15.81
1,17.60
2,20.29
3,22.79
4,23.44
5,21.66
6,18.86
7,16.58
8,15.41
9,15.41


In [13]:
df_3points["delta_theta"].sum()

328.99709495128616

In [23]:
def fix_angle(x):
    x = abs(x)
    if (x > 180):
        x = 360 - x
    return x

In [24]:
lat1 = df_3points["lat1"]
lat2 = df_3points["lat2"]
lat3 = df_3points["lat3"]
lon1 = df_3points["lon1"]
lon2 = df_3points["lon2"]
lon3 = df_3points["lon3"]

theta1 = np.arctan2(lat2 - lat1, (lon2 - lon1) * np.cos((lat1 + lat2) / 2))
theta2 = np.arctan2(lat3 - lat2, (lon3 - lon2) * np.cos((lat2 + lat3) / 2))
delta_theta = theta2 - theta1
np.rad2deg(delta_theta).apply(fix_angle)

0     15.85
1     17.61
2     20.26
3     22.77
4     23.41
5     21.64
6     18.86
7     16.59
8     15.40
9     15.40
10    16.59
11    18.86
12    21.64
13    23.41
14    22.77
15    20.26
16    17.61
dtype: float64